<a href="https://colab.research.google.com/github/skyshine460/music_genre/blob/main/submission_musicgenre_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
# df1:lightGBM #23
# df2:kNN #14
# df3:kNN+lightGBM #18

df1 = pd.read_csv("/content/drive/MyDrive/楽曲のジャンル推定チャレンジ！！（SOTA：SIGNATE Student Cup 2021春）/submit/submission_musicgenre_23.csv")
df2 = pd.read_csv("/content/drive/MyDrive/楽曲のジャンル推定チャレンジ！！（SOTA：SIGNATE Student Cup 2021春）/submit/submission_musicgenre_14.csv")
df3 = pd.read_csv("/content/drive/MyDrive/楽曲のジャンル推定チャレンジ！！（SOTA：SIGNATE Student Cup 2021春）/submit/submission_musicgenre_18.csv")

In [3]:
df1.columns = ['ID', 'col1']
df2.columns = ['ID', 'col2']
df3.columns = ['ID', 'col3']

In [4]:
df1.head()

,ID,col1
0,4047,10
1,4048,10
2,4049,8
3,4050,8
4,4051,7


In [5]:
df = pd.merge(df1, df2, on='ID', how='inner')
df = pd.merge(df, df3, on='ID', how='inner')

In [6]:
df.head()

,ID,col1,col2,col3
0,4047,10,10,10
1,4048,10,10,10
2,4049,8,8,8
3,4050,8,8,8
4,4051,7,7,7


In [7]:

# "submit" カラムを追加し、df1, df2, df3の多数決を取る
def majority_vote(row):
    # df1, df2, df3の各値をリストとして取り出す
    votes = [row['col1'], row['col2'], row['col3']]
    # 最も頻度が高い値を取得
    most_common = pd.Series(votes).mode().iloc[0]
    # 全てがバラバラ（多数決が決まらない）場合はdf3の値を採用
    if votes.count(most_common) == 1:
        return row['col3']
    return most_common

# 'submit' カラムに多数決結果を適用
df['submit'] = df.apply(majority_vote, axis=1)

# 結果を表示
df.head()


,ID,col1,col2,col3,submit
0,4047,10,10,10,10
1,4048,10,10,10,10
2,4049,8,8,8,8
3,4050,8,8,8,8
4,4051,7,7,7,7


In [8]:
df['col_1vs3'] = [1 if i == j else 0 for i, j in zip(df['col1'], df['col3'])]
df['col_2vs3'] = [1 if i == j else 0 for i, j in zip(df['col2'], df['col3'])]

print(df['col_1vs3'].sum()/len(df['col_1vs3']))
print(df['col_2vs3'].sum()/len(df['col_2vs3']))

0.8239802224969097
0.8986402966625463


In [9]:
df['col_3vsSubmit'] = [1 if i == j else 0 for i, j in zip(df['col3'], df['submit'])]

print(df['col_3vsSubmit'].sum()/len(df['col_3vsSubmit']))

0.9762669962917182


In [10]:
submit_df = df[['ID', 'submit']].reset_index(drop=True)
submit_df.head()

,ID,submit
0,4047,10
1,4048,10
2,4049,8
3,4050,8
4,4051,7


In [12]:
submit_df.to_csv("submission_musicgenre_24.csv", index=False, header=False)